In [108]:
import numpy as np
import pandas as pd
import os

## MLP engine

In [109]:
class MLP():
    def __init__(self, listOfLayer, activationFunction = "ReLU"):
        self.numLayers = len(listOfLayer)
        self.layers = [{} for i in range(self.numLayers)]
        self.activationFunction = activationFunction
        
        for i in range(1, self.numLayers):
            self.layers[i]["W"] = np.random.randn(listOfLayer[i], listOfLayer[i-1])  # weight (random)
            self.layers[i]["b"] = np.random.randn(listOfLayer[i], 1)  # bias (random)
            
        self.lossList = []  # loss value trace
        return
    
    def activation(self, z, actType="sigmoid"):
        if actType == "sigmoid":
            a = 1.0 / (1.0 + np.exp(-z))
        elif actType == "ReLU":
            a = np.where(z < 0.0, 0.0, z)  
            
        return a
    
    def inverseActivation(self, a, actType="sigmoid"):
        if actType == "sigmoid":
            dzda = a * (1.0 - a)
        elif actType == "ReLU":
            dzda = np.where(a > 0, 1.0, a)
            
        return dzda
    
    def forward(self, x, bs):
        self.layers[0]["a"] = np.copy(x)
        
        #Linear equation
        for i in range(1, self.numLayers):
            self.layers[i]["z"] = self.layers[i]["W"].dot(self.layers[i-1]["a"]) + self.layers[i]["b"]
            actType = "sigmoid" if i == self.numLayers - 1 else self.activationFunction
            self.layers[i]["a"] = self.activation(self.layers[i]["z"], actType)
            
        # last layer L does not use activation function    
        self.p = self.softmax(self.layers[-1]["a"])
        self.yHat = np.zeros(self.p.shape, dtype=int)  # predicted answers
        for j, i in enumerate(self.p.argmax(axis=0)):
            self.yHat[i, j] = 1
        return
    
    def softmax(self, a):
        expa = np.exp(a)
        return (expa / np.sum(expa, axis=0))
    
    def loss(self, y):
        return np.mean(- np.log(self.p) * y)
    
    def backprop(self, y, bs):
        # Derivative of loss function
        self.dJdp = - y / self.p
        
        # Derivative of softmax
        dpda = np.zeros((self.p.shape[0], self.p.shape[0], bs)) # a and p are of same dimension
        for b in range(bs):
            for i in range(dpda.shape[0]):
                for j in range(dpda.shape[1]):
                    dpda[i, j, b] = (self.p[i, b] - self.p[i, b] ** 2) if i == j else - self.p[i, b] * self.p[j, b]
        
        self.layers[-1]["dJda"] = np.zeros(self.layers[-1]["a"].shape)
        for b in range(bs):
            self.layers[-1]["dJda"][:, b] = dpda[:, :, b].dot(self.dJdp[:, b])
          
        for i in range(self.numLayers - 1, 0, -1):
            actType = "sigmoid" if i == self.numLayers - 1 else self.activationFunction
            self.layers[i]["dJdz"] = self.inverseActivation(self.layers[i]["a"], actType) * self.layers[i]["dJda"]
            self.layers[i]["dJdb"] = np.mean(self.layers[i]["dJdz"], axis = 1).reshape(self.layers[i]["b"].shape)
            self.layers[i]["dJdW"] = self.layers[i]["dJdz"].dot(self.layers[i-1]["a"].T) / bs
            self.layers[i-1]["dJda"] = self.layers[i]["W"].T.dot(self.layers[i]["dJdz"])
        return
    
    def update(self, lr):
        # lr: learning rate
        for i in range(1, self.numLayers):
            self.layers[i]["W"] -= lr * self.layers[i]["dJdW"]
            self.layers[i]["b"] -= lr * self.layers[i]["dJdb"]
        return
    
    def shuffle(self, a, b):
        shuffled_a = np.copy(a)
        shuffled_b = np.copy(b)
        permutation = np.random.permutation(a.shape[1])
        for oldindex, newindex in enumerate(permutation):
            shuffled_a[:, oldindex] = a[:, newindex]
            shuffled_b[:, oldindex] = b[:, newindex]
        return shuffled_a, shuffled_b
        
    def train(self, trainX, trainY, numEpoch=1, lr=0.01, bs=2):  
        for e in range(numEpoch):
            shuffled_trainX, shuffled_trainY = self.shuffle(trainX, trainY)
            for i in range(trainX.shape[1] // bs):
                x = shuffled_trainX[:, i*bs : (i+1)*bs]
                y = shuffled_trainY[:, i*bs : (i+1)*bs]
                self.forward(x, bs)
                self.lossList.append(self.loss(y))
                self.backprop(y, bs)
                self.update(lr)
                
        self.finalX = shuffled_trainX
        self.finalY = shuffled_trainY
        return      

## Data preprocessing

In [110]:
import pandas as pd
training = pd.read_csv("titanic/train.csv");

In [111]:
training.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [112]:
training.isna().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

In [113]:
np.random.seed(0)

training.drop(["Name", "PassengerId", "Ticket", "Cabin"], inplace=True, axis=1)
training.dropna(inplace = True)
x_train = training
rep_sex = {"male":0, "female":1}
rep_embarked = {"C":0, "Q":1, 'S':2}

x_train['Family'] = x_train ['SibSp'] + x_train['Parch']
x_train['IsAlone'] = 1
x_train['IsAlone'].loc[x_train['Family'] > 0] = 0
    
x_train.replace({"Sex": rep_sex, "Embarked": rep_embarked} , inplace = True )

print( x_train.shape )

(712, 10)


In [114]:
y = list(x_train.Survived)  # answer

In [115]:
training.drop(["Survived"], inplace=True, axis=1)

In [116]:
x_train[['Fare']] = x_train[['Fare']].round()

for col in x_train.columns[:]:
    x_train[col] = x_train[col] / x_train[col].max()

In [117]:
x_train.head()

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,Family,IsAlone
0,1.000000,0.0,0.2750,0.2,0.0,0.013672,1.0,0.142857,0.0
1,0.333333,1.0,0.4750,0.2,0.0,0.138672,0.0,0.142857,0.0
2,1.000000,1.0,0.3250,0.0,0.0,0.015625,1.0,0.000000,1.0
3,0.333333,1.0,0.4375,0.2,0.0,0.103516,1.0,0.142857,0.0
4,1.000000,0.0,0.4375,0.0,0.0,0.015625,1.0,0.000000,1.0


In [118]:
x_train.values

array([[1.  , 0.  , 0.28, ..., 1.  , 0.14, 0.  ],
       [0.33, 1.  , 0.47, ..., 0.  , 0.14, 0.  ],
       [1.  , 1.  , 0.33, ..., 1.  , 0.  , 1.  ],
       ...,
       [0.33, 1.  , 0.24, ..., 1.  , 0.  , 1.  ],
       [0.33, 0.  , 0.33, ..., 0.  , 0.  , 1.  ],
       [1.  , 0.  , 0.4 , ..., 0.5 , 0.  , 1.  ]])

## Training an neural network (NN)

In [119]:
input_x_train = np.array(x_train.values).T
input_y_train = np.zeros((2, len(y)), dtype=int)
for i in range(len(y)):
    input_y_train[y[i], i] = 1

In [120]:
nnTitanic = MLP([9, 23, 2], activationFunction = "sigmoid")
nnTitanic.train(input_x_train, input_y_train, numEpoch=7000, lr=0.25, bs=3)

In [121]:
nnTitanic.forward(input_x_train, 712)
predicted = nnTitanic.yHat.argmax(axis=0)

### Training Accuracy:

In [122]:
y_ = np.array(y)
print("Training Accuracy=", np.sum(y_ == predicted) / y_.shape[0])

Training Accuracy= 0.8595505617977528


## Testing data

In [146]:
# testing
testing = pd.read_csv("titanic/test.csv");
answer = pd.read_csv("titanic/gender_submission.csv")

testing = pd.concat([testing, answer], axis=1)
testing.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,PassengerId,Survived
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q,892,0
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S,893,1
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q,894,0
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S,895,0
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S,896,1


In [147]:
np.random.seed(0)
testing.drop([ "Name" , "PassengerId", "Ticket", "Cabin"], inplace = True, axis = 1 )
testing.dropna( inplace = True)
x_test = testing
repCol3 = {  "male":0, "female" : 1}
repCol8 = {"C" : 0 ,   "Q" : 1 , 'S' : 2  }

x_test['Family'] = x_test ['SibSp'] + x_test['Parch']
x_test['IsAlone'] = 1
x_test['IsAlone'].loc[x_test['Family'] > 0] = 0
    
x_test.replace({"Sex":repCol3, "Embarked":repCol8}, inplace = True)

print( x_test.shape )

(331, 10)


In [148]:
y_test = list(testing.Survived)
y_test = np.array(y_test)

testing.drop(["Survived"], inplace = True, axis = 1 )

In [149]:
x_test[['Fare']] = x_test[['Fare']].round()

for col in x_test.columns:
    x_test[col] = x_test[col]/x_test[col].max()
    
input_x_test = np.array(x_test.values).T

In [150]:
nnTitanic.forward(input_x_test, 331)
predicted_test = nnTitanic.yHat.argmax(axis=0)

### Testing Accuracy:

In [151]:
print("Testing Accuracy=", np.sum(y_test == predicted_test) / y_test.shape[0])

Testing Accuracy= 0.8277945619335347
